In [1]:
import cv2
import numpy as np
import time 

In [2]:

image = cv2.imread("left0192.jpg") # 根据路径读取一张图片
# 剪切有效区域
shapes=image.shape
# print(shapes)
# # image=image[0:int(1/3*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
image=image[int(1/5*shapes[1]):int(4/5*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
shapes=image.shape
# 计算有效区域的面积
propotion=image.shape[0]*image.shape[1]
# # 模糊处理
# # image = cv2.GaussianBlur(image, (7,7), 0)
cv2.imshow('image3',image)
# # image = cv2.pyrMeanShiftFiltering(image, 25, 10) 
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # BGR转HSV
# cv2.imshow('hsv',hsv)
end=image.copy()
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [3]:
# 绿色的范围
def getGreen(image):    
#     lower_green = np.array([35, 20, 150])
#     upper_green = np.array([70, 255, 255])
#     lower_green = np.array([16, 20, 200])
#     upper_green = np.array([36, 255, 255])
    lower_green = np.array([60, 40, 150])
    upper_green = np.array([95, 255, 255])
#     lower_green = np.array([0, 80, 145])
#     upper_green = np.array([8, 255, 255])
    return {'green':getRange(image,lower_green,upper_green)}

# 红色的范围
def getRed(image):    
#     low_red = np.array([0, 56, 90])
#     high_red = np.array([9, 203, 255])
    low_red = np.array([160, 80, 145])
    high_red = np.array([255, 255, 255])
    return {'red':getRange(image,low_red,high_red)}

# 绿色的范围
def getYellow(image):    
#     low_yellow = np.array([16, 20, 200])
#     high_yellow = np.array([36, 235, 255])
    low_yellow = np.array([15, 100, 125])
    high_yellow = np.array([30, 255, 255])
    return {'yellow':getRange(image,low_yellow,high_yellow)}

def getRange(image,low,high):
# cv2.inRange函数设阈值，去除背景部分
    mask = cv2.inRange(image, low, high)
    return mask

def deNoise(img):   
#灰度化
#     gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# 腐蚀    
    erode=cv2.erode(img,(100,100),iterations=4)
# 膨胀
    dilate=cv2.dilate(erode,(100,100),iterations=4)
#降噪（模糊处理用来减少瑕疵点）
    blur = cv2.blur(dilate, (5,5))
# 边缘检测    
    canny = cv2.Canny(blur, 150, 240)     
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (150, 150))
# 闭操作    
    closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)

    return closed

def getPositions(img):
#     cv2.imshow('image多点',img)
    ret,thresh = cv2.threshold(img,127,255,0)
    tmp,contours,hierarchy=cv2.findContours(thresh,1,2)
#     print('contours:',len(contours))
#     有效交通灯的比例
    if contours and cv2.contourArea(contours[0])/propotion >0.0005 and cv2.contourArea(contours[0])/propotion < 0.2:
#         print('--------')
        cnt=contours[0]    
        # 轮廓面积
        area=cv2.contourArea(cnt)
        # 轮廓周长
        perimeter = cv2.arcLength(cnt,True)
        # 轮廓近似
        epsilon=0.1*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
        k=cv2.isContourConvex(cnt)
        x,y,w,h=cv2.boundingRect(cnt)
#         cv2.rectangle(end,(x,y),(x+w,y+h),(0,255,0),2)
#         print('是否是闭合区域',k)
#         最小外切圆
        (x,y),radius = cv2.minEnclosingCircle(cnt)
        center = (int(x),int(y))
        radius = int(radius)
        cv2.circle(end,center,radius,(0,0,255),2)
#         res=image[int(y-radius):int(y+radius),int(x-radius):int(x+radius)]#已经剪切出目标区域
#         cv2.rectangle(end, (int(x-radius),int(y-radius)), (int(x+radius),int(y+radius)), (0, 255, 0), 2)
#         越界处理
        y0=int(y-radius)
        y1=int(y+radius)
        x0=int(x-radius)
        x1=int(x+radius)
        if y0 <0:
            y0=0
        if y1>shapes[0]:
            y1=shapes[0]
        if x0<0:
            x0=0
        if x1>shapes[1]:
            x1=shapes[1]
        cv2.imshow('endne',end)
        return [y0,y1,x0,x1]
#         return 1
    else:
        return 0
    


In [4]:
colors=[]
colors.append(getGreen(hsv))
colors.append(getRed(hsv))
colors.append(getYellow(hsv))
# 节省时间复杂度
start=time.clock()
# positions=[]
areas=[]
sumResult=0
_max={'color':0}
for i in range(len(colors)):
    closed = deNoise(list(colors[i].values())[0])
    coordinate = getPositions(closed)
#     position = {list(colors[i].keys())[0]:coordinate}
#     positions.append(position)
    if isinstance(coordinate,list):
        print('coordinate',coordinate)
#         cv2.imshow('endend',end)
        HSV=end[coordinate[0]:coordinate[1],coordinate[2]:coordinate[3],:]
#         cv2.imshow('HSV',HSV)
        HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)
        
        H, S, V = cv2.split(HSV)
        v = np.mean(V)
        print('v',v)
        print('ggg',list(colors[i].keys())[0])
        if v >list( _max.values())[0]:
            _max['color']=v
            _max[list(colors[i].keys())[0]] = _max.pop('color')
        sumResult+=v
        areas.append((list(colors[i].keys())[0],v))
    else:
        areas.append((list(colors[i].keys())[0],0))
        
# 按颜色值的大小排序

areas=sorted(areas,key=lambda x : (x[1]),reverse=True)
print(areas)
print(_max)
ended=time.clock()
print('total:',ended-start)
if int(areas[0][1]) != 0:
    print('The current color is:',areas[0][0])
else:
    print('没有交通灯')
cv2.waitKey(0)
cv2.destroyAllWindows()

[('green', 0), ('red', 0), ('yellow', 0)]
{'color': 0}
total: 3.4169014834518325
没有交通灯
